<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/cda-2021/blob/main/notebooks/cda_2021_05_25_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages("geepack")
install.packages("MuMIn")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(geepack) ## GEE models
library(MuMIn) ## for selecting GEE models
library(readxl) ## reading excel files 
library(tidyverse) ## processing
library(lubridate) ## processing date columns

Read excel data

In [4]:
beers <- read_excel("piwa.xlsx")
head(beers)

id,data,wielkosc_gosp,klm,wyksztalcenie,rok,miesiac,wojewodztwo,sklepy,sztuki
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
000208,20101105,3 Persons,Village,Profession Or Secondary,2010,11,Podkarpackie,Dyskont,1
000208,20101106,3 Persons,Village,Profession Or Secondary,2010,11,Podkarpackie,Sieci Detal,2
000208,20101122,3 Persons,Village,Profession Or Secondary,2010,11,Podkarpackie,Sieci Detal,1
000208,20101206,3 Persons,Village,Profession Or Secondary,2010,12,Podkarpackie,Dyskont,1
000208,20101212,3 Persons,Village,Profession Or Secondary,2010,12,Podkarpackie,Sieci Detal,2
000208,20101220,3 Persons,Village,Profession Or Secondary,2010,12,Podkarpackie,Dyskont,8


Processing of raw data

In [7]:
beers %>% 
  add_count(id, name = "times") %>%
  count(times)

times,n
<int>,<int>
1,567
2,758
3,921
4,1112
5,1120
6,1026
7,1176
8,1128
9,1233


In [8]:
beers_model <- beers %>%
  mutate(weekd = wday(ymd(data),week_start = 1)) %>%
  arrange(id, data) %>%
  add_count(id, name = "times")  %>%
  filter(times > 2, times <= 10)

dim(beers)
dim(beers_model)
head(beers_model)

[1] 80472    10

[1] 8936   12

id,data,wielkosc_gosp,klm,wyksztalcenie,rok,miesiac,wojewodztwo,sklepy,sztuki,weekd,times
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>
000386,20101221,2 Persons,City -19.999 Inhabitants,University,2010,12,Lubuskie,Dyskont,2,2,5
000386,20110502,2 Persons,City -19.999 Inhabitants,University,2011,5,Lubuskie,Hypermarkets,1,1,5
000386,20110604,2 Persons,City -19.999 Inhabitants,University,2011,6,Lubuskie,Supermarkets,4,6,5
000386,20110812,2 Persons,City -19.999 Inhabitants,University,2011,8,Lubuskie,Supermarkets,1,5,5
000386,20110818,2 Persons,City -19.999 Inhabitants,University,2011,8,Lubuskie,Supermarkets,1,4,5
000537,20100902,3 Persons,City -19.999 Inhabitants,Profession Or Secondary,2010,9,Dolnośląskie,Hypermarkets,8,4,4


First, fit first GEE model that assumes independence. This model is the same as quasi-Poisson model.

In [9]:
m1 <- geeglm(formula = sztuki ~ wielkosc_gosp + factor(weekd)+ klm + factor(miesiac) + sklepy + wyksztalcenie  + wojewodztwo, 
             data = beers_model,
             family  = poisson(), 
             corstr = "independence", ## R matrix from the materials (working correlation matrix)
             id = id, ## we need to sort data according to (id and) time!!!!
             scale.fix = FALSE ## means that we fit quasi-poisson model
             )
summary(m1)


Call:
geeglm(formula = sztuki ~ wielkosc_gosp + factor(weekd) + klm + 
    factor(miesiac) + sklepy + wyksztalcenie + wojewodztwo, family = poisson(), 
    data = beers_model, id = id, corstr = "independence", scale.fix = FALSE)

 Coefficients:
                                         Estimate    Std.err   Wald Pr(>|W|)
(Intercept)                             0.8359100  0.1649457 25.682 4.02e-07
wielkosc_gosp2 Persons                  0.2090599  0.0770536  7.361 0.006664
wielkosc_gosp3 Persons                  0.2219004  0.0704480  9.922 0.001634
wielkosc_gosp4 Persons                  0.2324044  0.0752256  9.545 0.002005
wielkosc_gosp5 Persons                  0.2637603  0.0861508  9.373 0.002201
wielkosc_gosp6 Persons                  0.3895593  0.1127865 11.930 0.000552
wielkosc_gosp7 Persons                  0.3896959  0.1453342  7.190 0.007332
wielkosc_gosp8 Persons                  0.2984589  0.2003329  2.220 0.136273
wielkosc_gosp9 Persons                  0.6822726  0.2132824 

This output refers to $\phi$ frm our materials.

```
  Estimate Std.err
(Intercept)    3.731  0.3394
Number of clusters:   1548  Maximum cluster size: 10 
```

We now assume that correlation may be modelled by using AR(1) process.

In [10]:
m2 <- geeglm(formula = sztuki ~ wielkosc_gosp + factor(weekd) + klm + 
               factor(miesiac) + sklepy + wyksztalcenie + wojewodztwo, 
             data = beers_model,
             family  = poisson(),
             corstr = "ar1", ## autocorrelation level 1
             id = id,
             scale.fix = FALSE)

summary(m2)          



Call:
geeglm(formula = sztuki ~ wielkosc_gosp + factor(weekd) + klm + 
    factor(miesiac) + sklepy + wyksztalcenie + wojewodztwo, family = poisson(), 
    data = beers_model, id = id, corstr = "ar1", scale.fix = FALSE)

 Coefficients:
                                       Estimate  Std.err  Wald Pr(>|W|)    
(Intercept)                             0.93061  0.17657 27.78  1.4e-07 ***
wielkosc_gosp2 Persons                  0.19772  0.08079  5.99  0.01439 *  
wielkosc_gosp3 Persons                  0.21581  0.07353  8.61  0.00334 ** 
wielkosc_gosp4 Persons                  0.21377  0.07681  7.74  0.00539 ** 
wielkosc_gosp5 Persons                  0.24717  0.08941  7.64  0.00570 ** 
wielkosc_gosp6 Persons                  0.37756  0.12998  8.44  0.00368 ** 
wielkosc_gosp7 Persons                  0.35786  0.18006  3.95  0.04688 *  
wielkosc_gosp8 Persons                  0.28189  0.21756  1.68  0.19510    
wielkosc_gosp9 Persons                  0.80770  0.29522  7.49  0.00622 ** 
fac

This output refers to the estimated $\phi$ and $\alpha$ from the materials.


```
Correlation structure = ar1 
Estimated Scale Parameters:

            Estimate Std.err
(Intercept)     3.77   0.341
  Link = identity 

Estimated Correlation Parameters:
      Estimate Std.err
alpha    0.554  0.0383
Number of clusters:   1548  Maximum cluster size: 10 
```


In [12]:
model.sel(m2, m1, rank = "QIC")

,(Intercept),factor(miesiac),factor(weekd),klm,sklepy,wielkosc_gosp,wojewodztwo,wyksztalcenie,family,corstr,df,qLik,QIC,delta,weight
,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<mdl.wght>
m1,0.836,+,+,+,+,+,+,+,poisson(log),independence,NA,9786,-19316,0.0,1.00e+00
m2,0.931,+,+,+,+,+,+,+,poisson(log),ar1,NA,9723,-19292,23.6,7.49e-06


Compare coefficients and standard errors

In [14]:
coef_table <- data.frame(model1 = coef(m1), model2 = coef(m2))
coef_table

,model1,model2
,<dbl>,<dbl>
(Intercept),0.835910,0.9306
wielkosc_gosp2 Persons,0.209060,0.1977
wielkosc_gosp3 Persons,0.221900,0.2158
wielkosc_gosp4 Persons,0.232404,0.2138
wielkosc_gosp5 Persons,0.263760,0.2472
wielkosc_gosp6 Persons,0.389559,0.3776
wielkosc_gosp7 Persons,0.389696,0.3579
wielkosc_gosp8 Persons,0.298459,0.2819
wielkosc_gosp9 Persons,0.682273,0.8077
